In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import requests
import pandas as pd

In [13]:
'''
driver = webdriver.Chrome()
driver.get("https://explore.openaire.eu/search/find/research-outcomes?foslabel=%220101%2520mathematics%257C%257Cmathematics%22")
input() #pause and wait for human input to solve captcha
el = driver.find_element(By.TAG_NAME, "search-result")

lis = el.find_elements(By.TAG_NAME, "li")

for li in lis:
    link = li.find_element(
        By.CSS_SELECTOR,
        "a.uk-link.uk-text-decoration-none.uk-width-expand"
    )
    if link.tag_name.lower() == "a":
        print(link.get_attribute("href"))
driver.close()
'''

https://explore.openaire.eu/search/publication?pid=10.1007%2Fs10455-005-1582-9
https://explore.openaire.eu/search/publication?pid=10.1016%2Fj.oxy.2022.07.001
https://explore.openaire.eu/search/publication?pid=10.1016%2Fj.jspi.2003.09.016
https://explore.openaire.eu/search/publication?pid=10.1080%2F00103629609369559
https://explore.openaire.eu/search/publication?pid=10.1016%2Fs0895-7177(97)80992-2
https://explore.openaire.eu/search/publication?pid=10.3208%2Fjgs.2.223
https://explore.openaire.eu/search/publication?pid=10.1007%2Fbf01386444
https://explore.openaire.eu/search/publication?pid=10.2478%2Famns.2022.2.0131
https://explore.openaire.eu/search/publication?pid=10.1016%2Fj.difgeo.2008.10.017
https://explore.openaire.eu/search/publication?pid=10.1007%2Fbf02754410


In [18]:
driver = webdriver.Chrome()
driver.get("https://explore.openaire.eu/search/find/research-outcomes?foslabel=%220101%2520mathematics%257C%257Cmathematics%22&page=1")
input() #pause and wait for human input to solve captcha

all_hrefs = set()   # use set to avoid duplicates
wait = WebDriverWait(driver, 10)

while len(all_hrefs) < 500:
    # Wait for search-result to update
    search_result = wait.until(
        EC.presence_of_element_located((By.TAG_NAME, "search-result"))
    )

    # Find <a> links inside it by class
    lis = search_result.find_elements(By.TAG_NAME, "li")

    for li in lis:
        link = li.find_element(
            By.CSS_SELECTOR,
            "a.uk-link.uk-text-decoration-none.uk-width-expand"
        )
        if link.tag_name.lower() == "a":
            href = link.get_attribute("href")
            if href:
                all_hrefs.add(href)

    print(f"Collected so far: {len(all_hrefs)}")

    if len(all_hrefs) >= 500:
        break

    # Find the Next button anywhere on page
    try:
        next_btn = wait.until(
            EC.element_to_be_clickable(
                (By.CSS_SELECTOR, "span.uk-link[aria-label='Next']")
            )
        )
        next_btn.click()
    except:
        print("No more pages or Next button missing!")
        break

    # Delay for network/DOM update
    time.sleep(2)

print("Done!")
print(f"Total links collected: {len(all_hrefs)}")
print(list(all_hrefs))


Collected so far: 10
No more pages or Next button missing!
Done!
Total links collected: 10
['https://explore.openaire.eu/search/publication?pid=10.1007%2Fbf02754410', 'https://explore.openaire.eu/search/publication?pid=10.1007%2Fbf01386444', 'https://explore.openaire.eu/search/publication?pid=10.1080%2F00103629609369559', 'https://explore.openaire.eu/search/publication?pid=10.1016%2Fj.difgeo.2008.10.017', 'https://explore.openaire.eu/search/publication?pid=10.2478%2Famns.2022.2.0131', 'https://explore.openaire.eu/search/publication?pid=10.1007%2Fs10455-005-1582-9', 'https://explore.openaire.eu/search/publication?pid=10.1016%2Fs0895-7177(97)80992-2', 'https://explore.openaire.eu/search/publication?pid=10.1016%2Fj.oxy.2022.07.001', 'https://explore.openaire.eu/search/publication?pid=10.1016%2Fj.jspi.2003.09.016', 'https://explore.openaire.eu/search/publication?pid=10.3208%2Fjgs.2.223']


In [10]:
driver = webdriver.Chrome()
driver.get("https://explore.openaire.eu/search/find/research-outcomes?sortBy=influence%20DESC&foslabel=%25220604%2520arts%257C%257Carts%2522&resultlanguagename=%2522English%2522")

input("Solve CAPTCHA then press Enter...")

wait = WebDriverWait(driver, 10)
all_hrefs = set()
page = 1

while len(all_hrefs) < 200:

    url = f"https://explore.openaire.eu/search/find/research-outcomes?sortBy=influence%20DESC&foslabel=%25220604%2520arts%257C%257Carts%2522&resultlanguagename=%2522English%2522&page={page}"
    print("Loading:", url)
    driver.get(url)
    time.sleep(10)

    # Wait for search-result to exist
    sr = wait.until(
        EC.presence_of_element_located((By.TAG_NAME, "search-result"))
    )

    lis = sr.find_elements(By.CSS_SELECTOR, "li")

    for li in lis:
        try:
            link = li.find_element(By.CSS_SELECTOR, "a.uk-link")
            
            # Wait until the href is populated
            wait.until(lambda d: link.get_attribute("href") is not None)
            
            href = link.get_attribute("href")
            if href:
                all_hrefs.add(href)
        except:
            pass

    print(f"Collected so far: {len(all_hrefs)}")

    # Stop if page returns nothing new
    if len(lis) == 0:
        print("No more results. Stopping.")
        break

    page += 1
    time.sleep(1)

print("DONE")
print(f"Total unique links: {len(all_hrefs)}")

Loading: https://explore.openaire.eu/search/find/research-outcomes?sortBy=influence%20DESC&foslabel=%25220604%2520arts%257C%257Carts%2522&resultlanguagename=%2522English%2522&page=1
Collected so far: 10
Loading: https://explore.openaire.eu/search/find/research-outcomes?sortBy=influence%20DESC&foslabel=%25220604%2520arts%257C%257Carts%2522&resultlanguagename=%2522English%2522&page=2
Collected so far: 20
Loading: https://explore.openaire.eu/search/find/research-outcomes?sortBy=influence%20DESC&foslabel=%25220604%2520arts%257C%257Carts%2522&resultlanguagename=%2522English%2522&page=3
Collected so far: 30
Loading: https://explore.openaire.eu/search/find/research-outcomes?sortBy=influence%20DESC&foslabel=%25220604%2520arts%257C%257Carts%2522&resultlanguagename=%2522English%2522&page=4
Collected so far: 40
Loading: https://explore.openaire.eu/search/find/research-outcomes?sortBy=influence%20DESC&foslabel=%25220604%2520arts%257C%257Carts%2522&resultlanguagename=%2522English%2522&page=5
Collec

In [11]:
series = pd.Series(list(all_hrefs))
series.to_csv("arts_links.csv")


In [7]:
series = pd.read_csv("./arts_links.csv")["0"]

series  

0      https://explore.openaire.eu/search/publication...
1      https://explore.openaire.eu/search/publication...
2      https://explore.openaire.eu/search/publication...
3      https://explore.openaire.eu/search/publication...
4      https://explore.openaire.eu/search/publication...
                             ...                        
195    https://explore.openaire.eu/search/publication...
196    https://explore.openaire.eu/search/publication...
197    https://explore.openaire.eu/search/publication...
198    https://explore.openaire.eu/search/publication...
199    https://explore.openaire.eu/search/publication...
Name: 0, Length: 200, dtype: object

In [12]:
series = pd.read_csv("./arts_links.csv")["0"]

driver = webdriver.Chrome()
driver.get("https://explore.openaire.eu/search/find/research-outcomes?sortBy=influence%20DESC&foslabel=%25220502%2520economics%2520and%2520business%257C%257Ceconomics%2520and%2520business%2522&resultlanguagename=%2522English%2522")

input("Solve CAPTCHA then press Enter...")

abstract_texts = []
group_label = []
titles = []
for link in series.tolist():
    print("Loading:", link)
    driver.get(link)
    time.sleep(4)

    try:
        abstract_parent = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(
                (By.ID, "summary")
            )
        )
        
        title_el = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((
                By.CSS_SELECTOR,
                "div.uk-text-break.multi-line-ellipsis span"
            ))
        )
        title = title_el.text.strip()
        
        abstract_element = abstract_parent.find_element(By.TAG_NAME, "p")
        
        abstract_text = driver.execute_script("return arguments[0].innerText;", abstract_element)
        
        print(title, abstract_text)
        
        abstract_texts.append(abstract_text)
        titles.append(title)
        group_label.append("Social & Humanities")

    except Exception as e:
        print("Abstract not found, skipping:", link)
        continue

df = pd.DataFrame({"group_label": group_label, "abstract": abstract_texts, "title": titles})
df


Loading: https://explore.openaire.eu/search/publication?pid=10.2307%2F3344994
Prediction of Performer Attentiveness Based on Rehearsal Activity and Teacher Behavior The purpose of this study was to examine videotaped teacher and performer behavior during several high school ensemble rehearsals to determine the predictability of a dependent variable, frequency of off-task behavior, by the following carrier variables: performance time; nonperformance time; frequency of social and academic approvals and disapprovals, stops, complete and incomplete teaching units, errors, and teacher eye contact. Subjects were six high school ensemble teachers and randomly selected students from two mixed choruses, three bands, and one orchestra. Videotapes, using two cameras and a special effects generator for a split-screen effect, were made for ex post facto analysis. The predictability of off task behavior carrier variables was ex amined, using a multiple regression analysis. The variability of off-tas

,group_label,abstract,title
0,Social & Humanities,The purpose of this study was to examine video...,Prediction of Performer Attentiveness Based on...
1,Social & Humanities,"Computer Music Journal, 26:4, pp. 31–43, Winte...",Max at Seventeen
2,Social & Humanities,ion Description and Purpose Variable names Pro...,Rethinking the Computer Music Language: SuperC...
3,Social & Humanities,The extensive range of sound sources admitted ...,The listening imagination: Listening in the el...
4,Social & Humanities,Abstract The style of a set of Swedish nursery...,Generative theories in language and music desc...
...,...,...,...
123,Social & Humanities,The Types of International Folktales: A Classi...,The Types of International Folktales: A Classi...
124,Social & Humanities,This article considers some ways in which the ...,"Popular music education in and for itself, and..."
125,Social & Humanities,The theoretical incoherence of the identity de...,Feeling Brown: Ethnicity and Affect in Ricardo...
126,Social & Humanities,1. This essay is about looking back: looking b...,Reinventing the Medium


In [13]:
df.to_csv("./arts.csv")

In [3]:
df

,group_label,abstract,title
0,Social & Humanities,A class of differentiated products is complete...,Hedonic Prices and Implicit Markets: Product D...
1,Social & Humanities,The attainment of quality in products and serv...,A Conceptual Model of Service Quality and Its ...
2,Social & Humanities,Abstract Nonsynchronous trading of securities ...,Estimating betas from nonsynchronous data
3,Social & Humanities,"Given incomplete factor markets, appropriate t...",Asset Stock Accumulation and Sustainability of...
4,Social & Humanities,This paper differs from previous examinations ...,Organizational Learning: The Contributing Proc...
...,...,...,...
175,Social & Humanities,Thls paper considers the prospects for constru...,On the mechanics of economic development
176,Social & Humanities,This study evaluated the sensitivity of maximu...,Fit indices in covariance structure modeling: ...
177,Social & Humanities,Organizational commitment has been conceptuali...,"The measurement and antecedents of affective, ..."
178,Social & Humanities,"In management contexts, mathematical programmi...",Some Models for Estimating Technical and Scale...


In [23]:
law = pd.read_csv("./law.csv", index_col=False)
bus = pd.read_csv("./bus-econ.csv", index_col=False)
df = pd.concat([law, bus], ignore_index=False)
df.drop(df.columns[0], axis=1, inplace=True)
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
df.drop_duplicates()
df

C:\Users\ASUS\AppData\Local\Temp\ipykernel_45580\1246943155.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


,group_label,abstract,title
0,Social & Humanities,THE likely routes of transmission of the acqui...,Transmission of the Human Immunodeficiency Virus
1,Social & Humanities,The authors identify and define key aspects of...,"Effectiveness, Transportability, and Dissemina..."
2,Social & Humanities,In response to the need for more sensitive ass...,Development of the severity of violence agains...
3,Social & Humanities,"Using a prospective cohorts design, a large sa...","Child abuse, neglect, and adult behavior: Rese..."
4,Social & Humanities,Antigay hate crimes (words or actions that are...,Hate crimes against lesbians and gay men: Issu...
...,...,...,...
151,Social & Humanities,Thls paper considers the prospects for constru...,On the mechanics of economic development
152,Social & Humanities,This study evaluated the sensitivity of maximu...,Fit indices in covariance structure modeling: ...
153,Social & Humanities,Organizational commitment has been conceptuali...,"The measurement and antecedents of affective, ..."
154,Social & Humanities,"In management contexts, mathematical programmi...",Some Models for Estimating Technical and Scale...
